<a href="https://colab.research.google.com/github/dejv07/sapssmath/blob/main/math_sapss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
import random

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///math_tasks.db'
app.config['JWT_SECRET_KEY'] = 'supersecretkey'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
jwt = JWTManager(app)

# Model uživatele
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(255), nullable=False)
    role = db.Column(db.String(10), nullable=False)  # 'student' nebo 'teacher'

# Model matematického příkladu
class MathProblem(db.Model):
    __tablename__ = 'math_problem'  # Explicitně pojmenuj tabulku
    id = db.Column(db.Integer, primary_key=True)
    difficulty = db.Column(db.String(10), nullable=False)  # 'easy', 'medium', 'hard'
    question = db.Column(db.Text, nullable=False)
    answer = db.Column(db.Text, nullable=False)
    created_by = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

# Model domácího úkolu
class Homework(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    assigned_to = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    problem_id = db.Column(db.Integer, db.ForeignKey('math_problem.id'), nullable=False)  # Opravený název tabulky
    status = db.Column(db.String(20), default='assigned')  # 'assigned', 'submitted', 'graded'
    student_answer = db.Column(db.Text, nullable=True)
    teacher_feedback = db.Column(db.Text, nullable=True)
    grade = db.Column(db.Integer, nullable=True)



@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password']).decode('utf-8')
    new_user = User(username=data['username'], password=hashed_password, role=data['role'])
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message': 'User created successfully'}), 201

@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    user = User.query.filter_by(username=data['username']).first()
    if user and bcrypt.check_password_hash(user.password, data['password']):
        access_token = create_access_token(identity={'username': user.username, 'role': user.role})
        return jsonify(access_token=access_token), 200
    return jsonify({'message': 'Invalid credentials'}), 401

@app.route('/math_problems', methods=['POST'])
@jwt_required()
def add_math_problem():
    current_user = get_jwt_identity()
    if current_user['role'] != 'teacher':
        return jsonify({'message': 'Unauthorized'}), 403

    data = request.get_json()
    problem = MathProblem(difficulty=data['difficulty'], question=data['question'], answer=data['answer'], created_by=User.query.filter_by(username=current_user['username']).first().id)
    db.session.add(problem)
    db.session.commit()
    return jsonify({'message': 'Problem added successfully'}), 201

@app.route('/math_problems', methods=['GET'])
@jwt_required()
def get_math_problems():
    problems = MathProblem.query.all()
    return jsonify([{'id': p.id, 'difficulty': p.difficulty, 'question': p.question, 'answer': p.answer} for p in problems]), 200

@app.route('/generate_homework', methods=['POST'])
@jwt_required()
def generate_homework():
    current_user = get_jwt_identity()
    if current_user['role'] != 'teacher':
        return jsonify({'message': 'Unauthorized'}), 403

    data = request.get_json()
    student = User.query.filter_by(username=data['student']).first()
    if not student or student.role != 'student':
        return jsonify({'message': 'Invalid student'}), 400

    problems = MathProblem.query.filter_by(difficulty=data['difficulty']).all()
    if not problems:
        return jsonify({'message': 'No problems found for this difficulty'}), 404

    selected_problems = random.sample(problems, min(len(problems), data.get('count', 5)))
    for problem in selected_problems:
        homework = Homework(assigned_to=student.id, problem_id=problem.id)
        db.session.add(homework)

    db.session.commit()
    return jsonify({'message': 'Homework assigned successfully'}), 201

@app.route('/submit_homework', methods=['POST'])
@jwt_required()
def submit_homework():
    current_user = get_jwt_identity()
    if current_user['role'] != 'student':
        return jsonify({'message': 'Unauthorized'}), 403

    data = request.get_json()
    homework = Homework.query.get(data['homework_id'])
    if not homework or homework.assigned_to != User.query.filter_by(username=current_user['username']).first().id:
        return jsonify({'message': 'Invalid homework'}), 400

    homework.student_answer = data['answer']
    homework.status = 'submitted'
    db.session.commit()
    return jsonify({'message': 'Homework submitted successfully'}), 200

@app.route('/grade_homework', methods=['POST'])
@jwt_required()
def grade_homework():
    current_user = get_jwt_identity()
    if current_user['role'] != 'teacher':
        return jsonify({'message': 'Unauthorized'}), 403

    data = request.get_json()
    homework = Homework.query.get(data['homework_id'])
    if not homework:
        return jsonify({'message': 'Invalid homework'}), 400

    homework.teacher_feedback = data['feedback']
    homework.grade = data['grade']
    homework.status = 'graded'
    db.session.commit()
    return jsonify({'message': 'Homework graded successfully'}), 200

if __name__ == '__main__':
    with app.app_context():
        db.create_all()
    app.run(debug=True)


ModuleNotFoundError: No module named 'flask_sqlalchemy'